# 영화 추천 시스템

- 캐글에서 사용되었던 영화 데이터셋을 가지고 영화 추천 시스템 만들기.

- TF-IDF와 코사인 유사도만으로 영화의 줄거리에 기반해서 영화를 추천하는 추천 시스템

- 다운로드 링크 : https://www.kaggle.com/rounakbanik/the-movies-dataset

## 0. 환경설정

### (1) 라이브러리 불러오기

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### (2) 데이터 불러오기

In [14]:
data = pd.read_csv("movies_metadata.csv", low_memory = False)
data.head(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## 1. 전처리

### (1) 결측값 확인

- overview 열의 결측값 확인

In [15]:
data['overview'].isna().value_counts()

overview
False    44512
True       954
Name: count, dtype: int64

- 결측값을 빈 값으로 채우기

In [16]:
data['overview'] = data['overview'].fillna('')

### (2) TF-IDF 행렬 생성

- 행렬 생성

In [24]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(45466, 75827)


- 행렬의 크기 출력

In [18]:
print(tfidf_matrix.shape)

(45466, 75827)


### (3) 코사인 유사도 구하기

In [19]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
# 코사인 유사도 연산 결과
print("코사인 유사도 연산 결과 : ", cosine_sim.shape)

코사인 유사도 연산 결과 :  (45466, 45466)


### (4) 영화의 타이틀과 인덱스를 딕셔너리에 매핑

In [20]:
title_to_index = dict(zip(data['title'], data.index))

- Test : "Father of the Bride Part II"라는 타이틀에 해당하는 인덱스 출력

In [21]:
print(title_to_index["Father of the Bride Part II"])

4


## 2. 유사한 영화를 찾아내는 클래스 생성

In [22]:
class get_recommendations :
    def __init__(self, title) :
        self.title = title
        
    def recommendation(self) :
        # 선택한 영화의 인덱스 추출
        idx = title_to_index[self.title]
        
        # 해당 영화와 모든 영화의 유사도 가져오기
        sim_scores = list(enumerate(cosine_sim[idx]))
        
        # 유사도에 따라 영화 정렬하기
        sim_scores.sort(key = lambda x : x[1], reverse = True)
        
        # 가장 유사한 10개의 인덱스 추출
        movie_indices = [line[0] for line in sim_scores[1:11]]
        
        # 가장 유사한 10개의 영화 제목 리턴
        return data['title'].iloc[movie_indices]

## 3. 추천 시스템 테스트

In [23]:
title = 'The Dark Knight Rises'

get_recommendations(title).recommendation()

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
15511                           Batman: Under the Red Hood
585                                                 Batman
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
18035                                     Batman: Year One
19792              Batman: The Dark Knight Returns, Part 1
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object